In [1]:
import pandas as pd
import numpy as np
import json
import keras
from keras.layers import Dense, LSTM
from keras.models import Sequential, load_model

/Users/giovanni/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open('../dati/lines.json', 'r') as f:
    data_lines = json.load(f)

with open('../dati/lines_2.json', 'r') as f:
    data_lines += json.load(f)

for elem in range(len(data_lines)):
    for arr in range(len(data_lines[elem])):
        tmp = []
        for f in range(7):
            tmp.append(data_lines[elem][arr][f])
        data_lines[elem][arr] = tmp

In [3]:
with open('../dati/circles.json', 'r') as f:
    data_circles = json.load(f)

for elem in range(len(data_circles)):
    for arr in range(len(data_circles[elem])):
        tmp = []
        for f in range(7):
            tmp.append(data_circles[elem][arr][f])
        data_circles[elem][arr] = tmp
        

In [4]:
with open('../dati/lines.json', 'r') as f:
    data_lines = json.load(f)

with open('../dati/lines_2.json', 'r') as f:
    data_lines += json.load(f)
    

def generator(data, labels):
    assert len(data) == len(labels)
    while True:
        for elem in range(len(data)):
            word_array = []
            for arr in range(len(data[elem])):
                tmp = []
                for f in range(7):
                    tmp.append(data[elem][arr][f])
                word_array.append(tmp)
            yield np.reshape(np.array(word_array), (1, len(word_array), 7)), [labels[elem]]

g_lines = generator(data_lines, [1] * len(data_lines))

In [5]:
with open('../dati/circles.json', 'r') as f:
    data_circles = json.load(f)
    
g_circles = generator(data_circles, [0] * len(data_circles))

In [6]:
with open('../dati/dario.json', 'r') as f:
    data_d = json.load(f)
g_dario = generator(data_d[:-3], [1] * 7)
g_dario_test = generator(data_d[-3:], [1] * 3)

In [7]:
with open('../dati/antonio.json', 'r') as f:
    data_a = json.load(f)
    
g_antonio = generator(data_a[:-3], [0] * 8)
g_antonio_test = generator(data_a[-3:], [0] * 3)

In [8]:
with open('../dati/taraz.json', 'r') as f:
    data_t = json.load(f)
    
g_taraz = generator(data_t[:-3], [0] * 7)
g_taraz_test = generator(data_t[-3:], [0] * 3)

In [9]:
with open('../dati/giovanni.json', 'r') as f:
    data_g = json.load(f)
    
g_giovanni = generator(data_g[:-3], [0] * 5)
g_giovanni_test = generator(data_g[-3:], [0] * 3)

In [10]:
def mix_generator(g1, g2):
    while True:
        yield next(g1)
        yield next(g2)

g_mix = mix_generator(g_lines, g_circles)
g_dario_giovanni = mix_generator(g_dario, g_giovanni)
g_dario_antonio = mix_generator(g_dario, g_antonio)
g_dario_taraz = mix_generator(g_dario, g_taraz)
g_dario_giovanni_antonio = mix_generator(g_dario_giovanni, g_dario_antonio)
g_dario_tutti = mix_generator(g_dario_giovanni_antonio, g_dario_taraz)

In [11]:
def generator_one_against_all(path_genuine, paths_impostors):
    with open(path_genuine, 'r') as f:
        genuine_data = json.load(f)
        
    train_ratio = round(len(genuine_data) / 5)
    assert len(genuine_data[:-train_ratio]) + len(genuine_data[-train_ratio:]) == len(genuine_data)
    genuine_generator = generator(genuine_data[:-train_ratio], [1] * (len(genuine_data) - train_ratio))
    genuine_generator_test = generator(genuine_data[-train_ratio:], [1] * train_ratio)
    
    generator_all = genuine_generator
    generator_all_test = genuine_generator_test
    
    for path in paths_impostors:
        with open(path, 'r') as f:
            impostor_data = json.load(f)
        train_ratio = round(len(impostor_data) / 5)
        assert len(impostor_data[:-train_ratio]) + len(impostor_data[-train_ratio:]) == len(impostor_data)
        generator_all = mix_generator(generator_all, generator(impostor_data[:-train_ratio], [0] * (len(impostor_data) - train_ratio)))
        generator_all_test = mix_generator(generator_all_test, generator(impostor_data[-train_ratio:], [0] * train_ratio))
        
    return generator_all, generator_all_test

impostors = ['../dati/giovanni.json', '../dati/taraz.json']
g_dario_impostors, g_dario_impostors_test = generator_one_against_all('../dati/dario.json', impostors)

In [12]:
model_dario = Sequential()
model_dario.add(LSTM(batch_size=1, input_shape=(None, 7), units=200, activation="sigmoid", return_sequences=False, recurrent_activation="hard_sigmoid"))
model_dario.add(Dense(1, activation='sigmoid'))

In [13]:
model_dario.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
hist = model_dario.fit_generator(g_dario_impostors, epochs=20, steps_per_epoch=42)

Epoch 1/20
42/42 [==============================] - 7s 161ms/step - loss: 0.5940 - acc: 0.7143
Epoch 2/20
42/42 [==============================] - 6s 144ms/step - loss: 0.4778 - acc: 0.7619
Epoch 3/20
42/42 [==============================] - 6s 144ms/step - loss: 0.4662 - acc: 0.7381
Epoch 4/20
42/42 [==============================] - 6s 143ms/step - loss: 0.4466 - acc: 0.7619
Epoch 5/20
42/42 [==============================] - 6s 145ms/step - loss: 0.4263 - acc: 0.7857
Epoch 6/20
42/42 [==============================] - 6s 146ms/step - loss: 0.3702 - acc: 0.8571
Epoch 7/20
42/42 [==============================] - 6s 143ms/step - loss: 0.4002 - acc: 0.8333
Epoch 8/20
42/42 [==============================] - 6s 144ms/step - loss: 0.3223 - acc: 0.9048
Epoch 9/20
42/42 [==============================] - 6s 145ms/step - loss: 0.3321 - acc: 0.8810
Epoch 10/20
42/42 [==============================] - 6s 144ms/step - loss: 0.3292 - acc: 0.8571
Epoch 11/20
42/42 [==============================

In [15]:
model_dario.evaluate_generator(g_dario_impostors_test, verbose=1, steps=6)

6/6 [==============================] - 0s 39ms/step


[0.4539774948110183, 0.8333333333333334]

In [ ]:
out_dario = model_dario.evaluate_generator(g_dario_test, verbose=1, steps=3)
out_giovanni = model_dario.evaluate_generator(g_giovanni_test, verbose=1, steps=3)
out_taraz = model_dario.evaluate_generator(g_taraz_test, verbose=1, steps=3)

out_dario, out_giovanni, out_taraz

3/3 [==============================] - 0s 24ms/step


In [ ]:
model_dario.save("model_dario.h5")
del model_dario

In [ ]:
model_dario_reloaded = load_model("model_dario.h5")

In [ ]:
out_dario_reloaded = model_dario_reloaded.evaluate_generator(g_dario_test, verbose=1, steps=3)

out_dario_reloaded

In [ ]:
# QUA SOTTO C'È ROBA VECCHIA

with open('lines.json', 'r') as f:
    data = json.load(f)
    
X_train = []
y_train = []
for i in range(len(data)):
    dfl = pd.DataFrame(data = data[i])
    for j in range(len(dfl)):
        X_train.append(dfl.iloc[j].values)
        y_train.append(0)           # lines
 

with open('circles.json', 'r') as f:
    data = json.load(f)

dfc = pd.DataFrame(data = data)    
for i in range(len(data)):
    dfc = pd.DataFrame(data = data[i])
    for j in range(len(dfc)):
        X_train.append(dfc.iloc[j].values)
        y_train.append(1)           # circles
    
    
with open('mix.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data = data)    
mix = []
for i in range(len(df)):
    mix.append(pd.DataFrame(data = data[i]))


X_train = pd.DataFrame(X_train).values
y_train1 = np.array(y_train[:158])
X_train = np.reshape(X_train, (1, X_train.shape[0], 7))

X_train1 = np.reshape(X_train1, (1, -1, 7))
y_train1 = np.reshape(y_train1, (1, y_train1.shape[0], 1))
print(X_train.shape)
print(y_train)

model = Sequential()
model.add(LSTM(batch_size=1, input_shape=(None, 7), units=200, activation="sigmoid", return_sequences=True, recurrent_activation="hard_sigmoid"))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(X_train1.shape)

hist = model.fit(X_train1, y_train1, batch_size=1, epochs=10, verbose=1)